# Experiment with transfer learning models b4 uploading to GCP

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from glob import glob
import matplotlib.pyplot as plt
import sklearn.neighbors
from imblearn.under_sampling import RandomUnderSampler
import imageio
from PIL import Image

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from skin_lesion_detection.data import get_data, clean_df, balance_nv, data_augmentation
from skin_lesion_detection.encoders import ImageScaler
from skin_lesion_detection.trainer import Trainer
from skin_lesion_detection.transfer_learning_models import TFModels

ModuleNotFoundError: No module named 'baseline_model'

In [4]:
df = get_data(50)

In [ ]:
df.shape

In [ ]:
df = clean_df(df)

In [ ]:
df.shape

In [ ]:
df = balance_nv(df, 5)

In [ ]:
df.shape

In [ ]:
df = data_augmentation(df)

In [ ]:
df.shape

In [ ]:
df.tail(10)